In [82]:
import pandas as pd
import csv
import re

In [27]:
file = pd.read_excel("Copy of Trade History.xlsx",header=1)
starting_point = pd.read_excel("Portfolio at end of 2009.xlsx",header=1)
file.head()
starting_point.head()




,Unnamed: 0,Trade Date,Sector,Ticker,Security Description,Total,Shares,Price,Comm,Fees,Settle Date,Principal,Trade Type
0,NaN,2009-12-31,CD,DIS,WALT DISNEY CO,-85462.50,2650,32.25,-8,0,2009-12-31,-85454.50,BUY
1,NaN,2009-12-31,CD,MCD,MCDONALD'S CORP,-79611.00,1275,62.44,-8,0,2009-12-31,-79603.00,BUY
2,NaN,2009-12-31,CD,AZO,AUTOZONE INC,-45840.30,290,158.07,-8,0,2009-12-31,-45832.30,BUY
3,NaN,2009-12-31,CD,VFC,VF CORP,-46287.68,632,73.24,-8,0,2009-12-31,-46279.68,BUY
4,NaN,2009-12-31,CD,LOW,LOWE'S COMPANIES INC,-58475.00,2500,23.39,-8,0,2009-12-31,-58467.00,BUY


In [38]:
#Makes the starting point portfolio which will be inputed below in make_portfolio
holdings = {}
for row in starting_point.itertuples():
    holdings[row[4]] = row[7]
print(holdings)    

#Fixing for now 
holdings.update({'FTR':"*"})
print(len(holdings))


{'DIS': 2650, 'MCD': 1275, 'AZO': 290, 'VFC': 632, 'LOW': 2500, 'KO': 1400, 'PM': 1550, 'CVS': 2750, 'RAH': 950, 'DEO': 1200, 'PG': 1600, 'APA': 1140, 'XOM': 1645, 'NE': 3990, 'SLB': 1030, 'CHK': 2925, 'SWN': 1538, 'HCBK': 3830, 'GS': 625, 'BLK': 285, 'MET': 1340, 'IVZ': 2230, 'JPM': 3308, 'CS': 1110, 'SIVB': 1200, 'GILD': 2185, 'ABT': 2050, 'TEVA': 2290, 'JNJ': 1050, 'HAE': 1500, 'WM': 1725, 'BUCY': 1850, 'NOC': 1250, 'UNP': 1225, 'HON': 2775, 'IBM': 1020, 'GOOGL': 460, 'CERN': 1560, 'APH': 2020, 'MANT': 1460, 'SNPS': 3295, 'SY': 2240, 'FCX': 538, 'BHP': 525, 'PPG': 1320, 'PX': 210, 'GEF': 672, 'CSCO': 1294, 'AMT': 800, 'VZ': 1980, 'OGE': 1000, 'NEE': 700, 'WEC': 1620}
54


In [32]:
#Simple EDA
file.describe()
file.head()
file.tail()
file.dtypes
file.columns

Index(['Unnamed: 0', 'Trade Date', 'Sector', 'Ticker', 'Security Description',
       'Total', 'Shares', 'Price', 'Comm', 'Fees', 'Settle Date', 'Principal',
       'Trade Type', 'Unnamed: 13', 'Unnamed: 14'],
      dtype='object')

In [35]:
cleaned = file[['Trade Date','Sector','Ticker','Security Description','Total','Shares','Price','Comm','Fees','Principal','Trade Type']].copy()
cleaned.columns

Index(['Trade Date', 'Sector', 'Ticker', 'Security Description', 'Total',
       'Shares', 'Price', 'Comm', 'Fees', 'Principal', 'Trade Type'],
      dtype='object')

In [36]:
cleaned.columns = ['trade_date','Sector', 'Ticker', 'Security Description', 'Total', 'Shares', 'Price','Comm', 'Fees', 'Principal', 'Trade Type']
cleaned['Shares']= cleaned['Shares'].astype(int)
cleaned.head(15)
cleaned.dtypes

trade_date              datetime64[ns]
Sector                          object
Ticker                          object
Security Description            object
Total                          float64
Shares                           int32
Price                          float64
Comm                           float64
Fees                           float64
Principal                      float64
Trade Type                      object
dtype: object

In [46]:
def make_portfolio(datatable,date):
    ts = pd.to_datetime(date)
    
    # Make the initial Portfoliol loaded in an an excel file name "Starting Point" above
    holdings = {}
    for row in starting_point.itertuples():
        holdings[row[4]] = row[7]   
        #Fixing for now 
    holdings.update({'FTR':"*"})
    
    #Create a copy of the trade history up to the provided date and execute transactions
    
    working = datatable[datatable.trade_date <= ts]

    for row in working.itertuples():
        type_change = row[11]
        shares = int(row[6])
        ticker = row[3]
        current_date = row[1]


        if type_change == "ADD":
            holdings[ticker] = int(holdings.get(ticker, "Error")) + shares 

        elif type_change == "SHAVE":
            holdings[ticker] = holdings.get(ticker,"Error") + shares


        elif type_change == "BUY":
            holdings.update({ticker:shares})

        elif type_change == "SELL":
            holdings.pop(ticker)

        else:
            print("error")

    

    print(len(holdings))
    print(holdings)
    return(holdings)

In [54]:
 a = make_portfolio(cleaned,'3/3/2011')

49
{'MCD': 740, 'PM': 1550, 'PG': 1600, 'GS': 727, 'TEVA': 1200, 'IBM': 1020, 'GOOGL': 460, 'APH': 2020, 'PPG': 1320, 'GEF': 672, 'AMT': 800, 'VZ': 1135, 'WEC': 1620, 'MJN': 1540, 'ANTM': 1100, 'JCI': 1550, 'GE': 5519, 'NVS': 2000, 'FISV': 2080, 'NFX': 1250, 'CELG': 1850, 'CME': 300, 'WFC': 4700, 'FLR': 1620, 'SYK': 1320, 'AEP': 920, 'COST': 1048, 'CB': 1545, 'EMC': 4000, 'CTL': 740, 'COP': 1252, 'GWW': 675, 'CSX': 1075, 'VALE': 1300, 'ITC': 655, 'CCL': 1233, 'PAA': 1125, 'VLO': 3050, 'TJX': 10160, 'HES': 1220, 'PB': 1720, 'ORCL': 3075, 'VIAB': 3150, 'MKC': 1450, 'BA': 1370, 'BHI': 1329, 'C': 15500, 'PRU': 1150, 'ON': 5925}


In [69]:
use = list(a.items())
df=pd.DataFrame(use,columns = ['Ticker','Shares'])
df = df.sort_values(by='Shares',ascending = False)
df.head()

,Ticker,Shares
46,C,15500
38,TJX,10160
48,ON,5925
16,GE,5519
22,WFC,4700


In [77]:
#Demonstrate how it will work
alpha = pd.DataFrame(data = {'a':[1,2,3,4] , 'b' : [1,2,3,4] , 'c' : [1,2,3,4]})
beta = pd.DataFrame(data = {'a':[1,2,3,4] , 'b' : [1,2,3,4] , 'c' : [1,2,3,4]})
print(alpha == beta)

      a     b     c
0  True  True  True
1  True  True  True
2  True  True  True
3  True  True  True


In [81]:
#Import the CapIQ print out
CapIQ = pd.read_excel("Cap IQ Test.xlsx")
CapIQ.head()
compare = CapIQ[['Ticker',"Shares"]].copy()
compare.head()

#Build pattern

,Ticker,Shares
0,NasdaqGS:GOOG.L,280.0
1,NYSE:MO,1850.0
2,NasdaqGS:AMZN,215.0
3,NYSE:AEP,700.0
4,NYSE:AMT,350.0
